In [3]:
for file in parquet_files:
    df = pd.read_parquet(file)
    print(f"\n{file.name}")
    print(dict(df.dtypes))



channels_snapshot.parquet
{'snapshot_date': dtype('O'), 'channel_id': <StringDtype(na_value=nan)>, 'subscriber_count': dtype('int64'), 'view_count': dtype('int64'), 'video_count': dtype('int64'), 'extracted_at': datetime64[us, UTC]}

channels_static.parquet
{'channel_id': <StringDtype(na_value=nan)>, 'channel_title': <StringDtype(na_value=nan)>, 'description': <StringDtype(na_value=nan)>, 'country': <StringDtype(na_value=nan)>, 'published_at': datetime64[us, UTC], 'thumbnail_url': <StringDtype(na_value=nan)>, 'channel_url': <StringDtype(na_value=nan)>, 'extracted_at': datetime64[us, UTC]}

playlist_items_manual_static.parquet
{'playlist_id': <StringDtype(na_value=nan)>, 'video_id': <StringDtype(na_value=nan)>, 'position': dtype('int64'), 'added_at': datetime64[us, UTC], 'extracted_at': datetime64[us, UTC]}

playlist_items_snapshot.parquet
{'snapshot_date': dtype('O'), 'playlist_id': <StringDtype(na_value=nan)>, 'video_id': <StringDtype(na_value=nan)>, 'position': dtype('int64'), 'adde

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

print("Proyecto desde .env:", os.getenv("GCP_PROJECT"))


from google.cloud import bigquery
import os

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"

client = bigquery.Client(project=PROJECT_ID)

print("Proyecto activo:", client.project)

datasets = list(client.list_datasets())
print("Datasets disponibles:")
for ds in datasets:
    print("-", ds.dataset_id)

Proyecto desde .env: youtube-datasets-360
Proyecto activo: youtube-datasets-360
Datasets disponibles:
- angelgarciadatablog


In [2]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

parquet_files = sorted(PROCESSED_PATH.glob("*.parquet"))

print(f"\n📦 Auditoría de Parquets ({len(parquet_files)} archivos)\n")

results = []

for file in parquet_files:
    df = pd.read_parquet(file)
    
    rows = len(df)
    cols = len(df.columns)
    
    if "snapshot_date" in df.columns:
        unique_dates = df["snapshot_date"].nunique()
        snapshot_status = f"{unique_dates} fechas"
    else:
        unique_dates = "-"
        snapshot_status = "STATIC"
    
    results.append({
        "archivo": file.name,
        "filas": rows,
        "columnas": cols,
        "snapshot_info": snapshot_status
    })

audit_df = pd.DataFrame(results)
audit_df



📦 Auditoría de Parquets (7 archivos)



,archivo,filas,columnas,snapshot_info
0,channels_snapshot.parquet,1,6,1 fechas
1,channels_static.parquet,1,8,STATIC
2,playlist_items_manual_static.parquet,165,5,STATIC
3,playlist_items_snapshot.parquet,165,6,1 fechas
4,playlists_manual_static.parquet,16,8,STATIC
5,videos_snapshot.parquet,199,9,1 fechas
6,videos_static.parquet,199,10,STATIC
